In [13]:
import pandas as pd
import numpy as np
import os
import json
from pathlib import Path

from archetypal import IDF, parallel_process
from archetypal import BuildingTemplate
from archetypal.umi_template import UmiTemplateLibrary
from archetypal.schedule import Schedule

from geomeppy.patches import EpBunch

from umi.ubem import init_template
from utils.schedules import get_schedules
from utils.constants import *

ref_buildings_path = "D:/Users/zoelh/Dropbox (MIT)/Thesis Work/Building Energy References/ReferenceBuildings"

# Load all reference templates

In [14]:
reference_names = [x for x in os.listdir(ref_buildings_path) if "." not in x]

full_reference_paths = [
    Path(ref_buildings_path, x, x + "New2004_v1.4_7.2_1A_USA_FL_MIAMI.idf")
    for x in reference_names
]

reference_paths = [x for x in os.listdir(ref_buildings_path) if "MIAMI.idf" in x]

In [15]:
zones = ["Perimeter"]
schedules_from_reference_umi = {}
for i, template_path in enumerate(full_reference_paths):
    idf = IDF(template_path)
    print(idf.name)
    scheds = idf._get_used_schedules()
    # print([x for x in scheds if "OCC" in x.upper()])
    # print([x for x in scheds if "LIGHT" in x.upper()])
    # print([x for x in scheds if "EQUIP" in x.upper()])
    sched_dict = {}
    for name in scheds:
        if "EQUIP" in name.upper() or "EQP" in name.upper():
            sched_dict[name] = Schedule.from_epbunch(idf.get_schedule_epbunch(name))
        if "LIGHT" in name.upper():
            sched_dict[name] = Schedule.from_epbunch(idf.get_schedule_epbunch(name))
        if "OCC" in name.upper():
            sched_dict[name] = Schedule.from_epbunch(idf.get_schedule_epbunch(name))
    schedules_from_reference_umi[idf.name] = sched_dict


RefBldgFullServiceRestaurantNew2004_v1.4_7.2_1A_USA_FL_MIAMI.idf
RefBldgHospitalNew2004_v1.4_7.2_1A_USA_FL_MIAMI.idf
RefBldgLargeHotelNew2004_v1.4_7.2_1A_USA_FL_MIAMI.idf
RefBldgLargeOfficeNew2004_v1.4_7.2_1A_USA_FL_MIAMI.idf
RefBldgMediumOfficeNew2004_v1.4_7.2_1A_USA_FL_MIAMI.idf
RefBldgMidriseApartmentNew2004_v1.4_7.2_1A_USA_FL_MIAMI.idf
RefBldgOutPatientNew2004_v1.4_7.2_1A_USA_FL_MIAMI.idf
RefBldgPrimarySchoolNew2004_v1.4_7.2_1A_USA_FL_MIAMI.idf
RefBldgQuickServiceRestaurantNew2004_v1.4_7.2_1A_USA_FL_MIAMI.idf
RefBldgSecondarySchoolNew2004_v1.4_7.2_1A_USA_FL_MIAMI.idf
RefBldgSmallHotelNew2004_v1.4_7.2_1A_USA_FL_MIAMI.idf
RefBldgSmallOfficeNew2004_v1.4_7.2_1A_USA_FL_MIAMI.idf
RefBldgStand-aloneRetailNew2004_v1.4_7.2_1A_USA_FL_MIAMI.idf
RefBldgStripMallNew2004_v1.4_7.2_1A_USA_FL_MIAMI.idf
RefBldgSuperMarketNew2004_v1.4_7.2_1A_USA_FL_MIAMI.idf
RefBldgWarehouseNew2004_v1.4_7.2_1A_USA_FL_MIAMI.idf


In [16]:
schedule_names = [
    "EquipmentAvailabilitySchedule",
    "LightsAvailabilitySchedule",
    "OccupancySchedule",
]

reference_schedules = np.load("./data/schedules.npy")
reference_schedules.shape

(16, 3, 8760)

In [17]:
names_in_order = {}
for t in range(16):
    d = {}
    for s, name in enumerate(schedule_names):
        for ref_path, scheds in schedules_from_reference_umi.items():
            for _, template_sched in scheds.items():
                npy_sched = Schedule.from_values(name, reference_schedules[t, s, :])
                if np.array_equal(npy_sched.Values, template_sched.Values):
                    # names_in_order.append(ref_path)
                    d[name] = str(ref_path).split("New2004_")[0]
    names_in_order[t] = d

In [18]:
names_in_order

{0: {'EquipmentAvailabilitySchedule': 'RefBldgWarehouse',
  'LightsAvailabilitySchedule': 'RefBldgWarehouse',
  'OccupancySchedule': 'RefBldgWarehouse'},
 1: {'EquipmentAvailabilitySchedule': 'RefBldgLargeHotel',
  'LightsAvailabilitySchedule': 'RefBldgSmallHotel',
  'OccupancySchedule': 'RefBldgLargeHotel'},
 2: {'EquipmentAvailabilitySchedule': 'RefBldgLargeHotel',
  'LightsAvailabilitySchedule': 'RefBldgQuickServiceRestaurant',
  'OccupancySchedule': 'RefBldgFullServiceRestaurant'},
 3: {'EquipmentAvailabilitySchedule': 'RefBldgQuickServiceRestaurant',
  'LightsAvailabilitySchedule': 'RefBldgQuickServiceRestaurant',
  'OccupancySchedule': 'RefBldgQuickServiceRestaurant'},
 4: {'EquipmentAvailabilitySchedule': 'RefBldgStripMall',
  'LightsAvailabilitySchedule': 'RefBldgStripMall',
  'OccupancySchedule': 'RefBldgStripMall'},
 5: {'EquipmentAvailabilitySchedule': 'RefBldgSuperMarket',
  'LightsAvailabilitySchedule': 'RefBldgSuperMarket',
  'OccupancySchedule': 'RefBldgSuperMarket'},
 6

In [31]:
names_in_order.update(
    {
        1: {
            "EquipmentAvailabilitySchedule": "RefBldgLargeHotel",
            "LightsAvailabilitySchedule": "RefBldgLargeHotel",
            "OccupancySchedule": "RefBldgLargeHotel",
        },
        2: {
            "EquipmentAvailabilitySchedule": "RefBldgFullServiceRestaurant",
            "LightsAvailabilitySchedule": "RefBldgFullServiceRestaurant",
            "OccupancySchedule": "RefBldgFullServiceRestaurant",
        },
        12: {
            "EquipmentAvailabilitySchedule": "RefBldgPrimarySchool",
            "LightsAvailabilitySchedule": "RefBldgPrimarySchool",
            "OccupancySchedule": "RefBldgPrimarySchool",
        },
        14: {
            "EquipmentAvailabilitySchedule": "AlwaysOff",
            "LightsAvailabilitySchedule": "AlwaysOff",
            "OccupancySchedule": "AlwaysOff",
        },
        15: {
            "EquipmentAvailabilitySchedule": "AlwaysOn",
            "LightsAvailabilitySchedule": "AlwaysOn",
            "OccupancySchedule": "AlwaysOn",
        },
    }
)

In [32]:
names_in_order

{0: {'EquipmentAvailabilitySchedule': 'RefBldgWarehouse',
  'LightsAvailabilitySchedule': 'RefBldgWarehouse',
  'OccupancySchedule': 'RefBldgWarehouse'},
 1: {'EquipmentAvailabilitySchedule': 'RefBldgLargeHotel',
  'LightsAvailabilitySchedule': 'RefBldgLargeHotel',
  'OccupancySchedule': 'RefBldgLargeHotel'},
 2: {'EquipmentAvailabilitySchedule': 'RefBldgFullServiceRestaurant',
  'LightsAvailabilitySchedule': 'RefBldgFullServiceRestaurant',
  'OccupancySchedule': 'RefBldgFullServiceRestaurant'},
 3: {'EquipmentAvailabilitySchedule': 'RefBldgQuickServiceRestaurant',
  'LightsAvailabilitySchedule': 'RefBldgQuickServiceRestaurant',
  'OccupancySchedule': 'RefBldgQuickServiceRestaurant'},
 4: {'EquipmentAvailabilitySchedule': 'RefBldgStripMall',
  'LightsAvailabilitySchedule': 'RefBldgStripMall',
  'OccupancySchedule': 'RefBldgStripMall'},
 5: {'EquipmentAvailabilitySchedule': 'RefBldgSuperMarket',
  'LightsAvailabilitySchedule': 'RefBldgSuperMarket',
  'OccupancySchedule': 'RefBldgSuperMa

In [33]:
index = []
for _, item in names_in_order.items():
    for name, ref in item.items():
        index.append((ref, name))

In [46]:
# reshaped_refs = np.reshape(reference_schedules, (48, 8760))
reshaped_refs = []
for i in range(16):
    reshaped_refs.extend(reference_schedules[i])
reshaped_refs = np.array(reshaped_refs)
reshaped_refs.shape

(48, 8760)

In [47]:
schedules_df = pd.DataFrame(
    reshaped_refs, index=pd.MultiIndex.from_tuples(index)
)
print(schedules_df.shape)
schedules_df

(48, 8760)


0       1     \
RefBldgWarehouse              EquipmentAvailabilitySchedule  0.1000  0.1000   
                              LightsAvailabilitySchedule     0.1000  0.1000   
                              OccupancySchedule              0.0000  0.0000   
RefBldgLargeHotel             EquipmentAvailabilitySchedule  0.1000  0.1000   
                              LightsAvailabilitySchedule     0.3000  0.3000   
                              OccupancySchedule              0.1000  0.1000   
RefBldgFullServiceRestaurant  EquipmentAvailabilitySchedule  0.1000  0.1000   
                              LightsAvailabilitySchedule     0.4500  0.1500   
                              OccupancySchedule              0.0500  0.0000   
RefBldgQuickServiceRestaurant EquipmentAvailabilitySchedule  0.1000  0.1000   
                              LightsAvailabilitySchedule     0.4500  0.1500   
                              OccupancySchedule              0.0500  0.0000   
RefBldgStripMall              EquipmentAvailabilitySchedule  0.1500  0.1500   
                              LightsAvailabilitySchedule     0.0500  0.0500   
                              OccupancySchedule              0.0000  0.0000   
RefBldgSuperMarket            EquipmentAvailabilitySchedule  0.1500  0.1500   
                              LightsAvailabilitySchedule     0.0500  0.0500   
                              OccupancySchedule              0.0000  0.0000   
RefBldgSmallOffice            EquipmentAvailabilitySchedule  0.3000  0.3000   
                              LightsAvailabilitySchedule     0.0500  0.0500   
                              OccupancySchedule              0.0000  0.0000   
RefBldgMediumOffice           EquipmentAvailabilitySchedule  0.3000  0.3000   
                              LightsAvailabilitySchedule     0.0500  0.0500   
                              OccupancySchedule              0.0000  0.0000   
RefBldgOutPatient             EquipmentAvailabilitySchedule  0.3000  0.3000   
                              LightsAvailabilitySchedule     0.0500  0.0500   
                              OccupancySchedule              0.0000  0.0000   
RefBldgHospital               EquipmentAvailabilitySchedule  0.3000  0.3000   
                              LightsAvailabilitySchedule     0.0500  0.0500   
                              OccupancySchedule              0.0000  0.0000   
RefBldgMidriseApartment       EquipmentAvailabilitySchedule  0.3300  0.3300   
                              LightsAvailabilitySchedule     0.1800  0.1800   
                              OccupancySchedule              0.0000  0.0000   
RefBldgSecondarySchool        EquipmentAvailabilitySchedule  0.3500  0.3500   
                              LightsAvailabilitySchedule     0.1773  0.1773   
                              OccupancySchedule              0.0000  0.0000   
RefBldgPrimarySchool          EquipmentAvailabilitySchedule  0.3500  0.3500   
                              LightsAvailabilitySchedule     0.1773  0.1773   
                              OccupancySchedule              0.0000  0.0000   
RefBldgSmallHotel             EquipmentAvailabilitySchedule  0.4000  0.4000   
                              LightsAvailabilitySchedule     0.3000  0.3000   
                              OccupancySchedule              0.0000  0.0000   
AlwaysOff                     EquipmentAvailabilitySchedule  0.0000  0.0000   
                              LightsAvailabilitySchedule     0.0000  0.0000   
                              OccupancySchedule              0.0000  0.0000   
AlwaysOn                      EquipmentAvailabilitySchedule  1.0000  1.0000   
                              LightsAvailabilitySchedule     1.0000  1.0000   
                              OccupancySchedule              1.0000  1.0000   

                                                               2       3     \
RefBldgWarehouse              EquipmentAvailabilitySchedule  0.1000  0.1000   
                              Lig

In [44]:
schedules_df.to_hdf("data/schedules.h5", key="df")

In [45]:
schedules_df.xs('EquipmentAvailabilitySchedule', level=1, drop_level=True).to_csv("EquipmentAvailabilitySchedule.csv")
schedules_df.xs('LightsAvailabilitySchedule', level=1, drop_level=True).to_csv("LightsAvailabilitySchedule.csv")
schedules_df.xs('OccupancySchedule', level=1, drop_level=True).to_csv("OccupancySchedule.csv")